# Part (a)

We will represent the new schedule as a 3D matrix X where $X_{ijk} = 1$ if department $i$ is assigned to room $j$ on day $k$.

We want to minimze the total under-allocation of hours as we transfer from the old schedule to the new schedule. The variable $s_i$ represents the under-allocation for each department, while the variable $t_i$ represents the ideal time that would result in a perfect scaling of time from the old schedule to the new schedule, which is impossible due to room and day restrictions. The variable $a_i$ represents the total time assigned to department $i$, which is determined by the total duration of room $i$ on day $k$ ($d_{jk}$) and which department holds which room on which days ($x_{ijk}$).

\begin{align}
{\text{min}} \;\; & \sum_{i=1}^6 \frac{s_i}{t_i} \\
\text{s.t. } & s_i \geq t_i - a_i, \\
& a_i = \sum_{j = 1}^5 \sum_{k = 1}^5 d_{jk} * x_{ijk}, \\
& s_i \geq 0, \\
& \sum_{i = 1}^6 x_{ijk} = 1, \\
\end{align}

In [19]:
from gurobipy import *
import numpy as np

# initialize target and duration vectors
total_hrs = 213.5
percents = np.array([0.484, 0.042, 0.253, 0.074, 0.053, 0.095])
t = total_hrs * percents
d = np.array([[9, 9, 9, 9, 7.5],
              [9, 9, 9, 9, 7.5],
              [9, 9, 9, 9, 7.5],
              [9, 9, 9, 9, 7.5],
              [9, 8, 8, 8, 6.5]])

# initialize model
mod_a = Model()

# set ranges for iteration
m = 6 # range for i (dept)
n = 5 # range for j (rooms)
r = 5 # range for k (days)

# initialize 3D matrix to represent schedule
x = mod_a.addVars(m, n, r, vtype = GRB.BINARY)
# initialize vector of assigned hours
a = mod_a.addVars(m)
# initialize under-allocation vector
s = mod_a.addVars(m)

# compute actual hours assigned
for i in range(m):
    mod_a.addConstr(a[i] == sum(sum(d[k][j] * x[i, j, k] for j in range(n)) for k in range(r)))

# constraint on s vector values
for i in range(m):
    mod_a.addConstr(s[i] >= t[i] - a[i])

# only penalize under-allocation
for i in range(m):
    mod_a.addConstr(s[i] >= 0.0)
        
# limit each room to 1 department per day
for j in range(n):
    for k in range(r):
        mod_a.addConstr(sum(x[i, j, k] for i in range(m)) <= 1)

# contruct objective
mod_a.setObjective(sum(s[i] / t[i] for i in range(m)), GRB.MINIMIZE)


In [20]:
# update and solve the model
mod_a.update()
mod_a.optimize()


Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 43 rows, 162 columns and 324 nonzeros
Model fingerprint: 0xac70c4e1
Variable types: 12 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 6.0000000
Found heuristic solution: objective 6.0000000
Presolve removed 12 rows and 6 columns
Presolve time: 0.00s
Presolved: 31 rows, 156 columns, 306 nonzeros
Variable types: 4 continuous, 152 integer (150 binary)

Root relaxation: objective 2.066116e-03, 58 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00207    0    8    6.00000    0.00207   100%     -    0s
H    0     0 

In [21]:
# retrieve optimal value and optimal solution
opt_val = mod_a.objval
print("Optimal cost:", opt_val)


Optimal cost: 0.05190597648318144


In [22]:
# dsplay x matrix
x_opt = [x[i, j, k].x for i in range(m) for j in range(n) for k in range(r)]
print(x_opt)


[-0.0, 1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, -0.0, 1.0, 0.0, 1.0, 1.0, -0.0, -0.0, -0.0, 1.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, 1.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, 0.0, -0.0, -0.0, 1.0, 0.0, -0.0, 0.0, 1.0, 0.0, 1.0, -0.0, 0.0, -0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, 1.0, 0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, 0.0, 0.0, -0.0, 1.0, -0.0, -0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, 1.0, 1.0, 0.0]


In [23]:
# display penalty vector s
s_opt = [s[i].x for i in range(m)]
s_opt


[5.333999999999992, 0.0, 0.015500000000018943, 0.0, 0.0, 0.0]

# Part (b)

Same problem, but we now incorporate 3 floors (indexed by $l$) with rooms 1 and 2 on floor 1, rooms 3 and 4 on floor 2, and room 5 on floor 3. We don't want to split up departments onto different floors.

We initialize a matrix $v$ where $v_{ikl}  = 1$ if departement $i$ is on floor $l$ on day $k$. Then, we just have to modify the previous model by adding the following constraints to account for the floors:


\begin{align}
& 2v_{1k1} \geq x_{i1k} + x_{i2k}, \\
& 2v_{1k2} \geq x_{i3k} + x_{i4k}, \\
& v_{1k1} \geq x_{i5k}, \\
& \sum_{l = 1}^3 v_{ikl} \leq 1
\end{align}

In [24]:
from gurobipy import *
import numpy as np

# initialize target and duration vectors
total_hrs = 213.5
percents = np.array([0.484, 0.042, 0.253, 0.074, 0.053, 0.095])
t = total_hrs * percents
d = np.array([[9, 9, 9, 9, 7.5],
              [9, 9, 9, 9, 7.5],
              [9, 9, 9, 9, 7.5],
              [9, 9, 9, 9, 7.5],
              [9, 8, 8, 8, 6.5]])

# initialize model
mod_b = Model()

# set ranges for iteration
m = 6 # range for i (dept)
n = 5 # range for j (rooms)
r = 5 # range for k (days)
u = 3 # range for l (floors)

# initialize 3D matrix to represent schedule
x = mod_b.addVars(m, n, r, vtype = GRB.BINARY)
# initialize vector of assigned hours
a = mod_b.addVars(m)
# initialize under-allocation vector
s = mod_b.addVars(m)
# initialize floor tracker
v = mod_b.addVars(m, r, u, vtype = GRB.BINARY)

# compute actual hours assigned
for i in range(m):
    mod_b.addConstr(a[i] == sum(sum(d[k][j] * x[i, j, k] for j in range(n)) for k in range(r)))

# constraint on s vector values
for i in range(m):
    mod_b.addConstr(s[i] >= t[i] - a[i])

# only penalize under-allocation
for i in range(m):
    mod_b.addConstr(s[i] >= 0.0)
        
# limit each room to 1 department per day
for j in range(n):
    for k in range(r):
        mod_b.addConstr(sum(x[i, j, k] for i in range(m)) <= 1)
        
# don't separate departments by floor
for k in range(r):
    for i in range(m):
        mod_b.addConstr(sum(v[i, k, l] for l in range(u)) <= 1)
        
# associate rooms with floors
for i in range(m):
    for k in range(r):
        mod_b.addConstr(2 * v[i, k, 0] >= x[i, 0, k] + x[i, 1, k])
        mod_b.addConstr(2 * v[i, k, 1] >= x[i, 2, k] + x[i, 3, k])
        mod_b.addConstr(v[i, k, 2] >= x[i, 4, k])

# contruct objective
mod_b.setObjective(sum(s[i] / t[i] for i in range(m)), GRB.MINIMIZE)


In [25]:
# update and solve the model
mod_b.update()
mod_b.optimize()


Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 163 rows, 252 columns and 654 nonzeros
Model fingerprint: 0x471cd3c4
Variable types: 12 continuous, 240 integer (240 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 6.0000000
Found heuristic solution: objective 6.0000000
Presolve removed 72 rows and 66 columns
Presolve time: 0.00s
Presolved: 91 rows, 186 columns, 516 nonzeros
Variable types: 4 continuous, 182 integer (180 binary)

Root relaxation: objective 1.387152e-01, 83 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.13872    0    4    6.00000    0.13872  97.7%     -    0s
H    0     

In [26]:
# retrieve optimal value and optimal solution
opt_val = mod_b.objval
print("Optimal cost:", opt_val)


Optimal cost: 0.1387152340952639


In [27]:
# dsplay x matrix
x_opt = [x[i, j, k].x for i in range(m) for j in range(n) for k in range(r)]
print(x_opt)


[1.0, -0.0, -0.0, -0.0, 1.0, 1.0, -0.0, -0.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, 0.0, -0.0, -0.0, -0.0, 1.0, 1.0, -0.0, -0.0, -0.0, 1.0, -0.0, 1.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0, 0.0, 1.0, -0.0, 1.0, 1.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]


In [28]:
# display v matrix
v_opt = [v[i, k, l].x for i in range(m) for k in range(r) for l in range(u)]
print(v_opt)


[1.0, -0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, -0.0, 0.0, 1.0, -0.0, 0.0, 1.0, -0.0, 0.0, 1.0, -0.0, 0.0, 1.0, -0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, -0.0, 1.0, 0.0, 0.0, 1.0, 1.0, -0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, -0.0, 0.0, 1.0, -0.0, 0.0, 0.0, -0.0, 1.0, 0.0, 1.0, 0.0, 0.0, -0.0, 1.0, 1.0, -0.0, 0.0, 1.0, -0.0, 0.0, 1.0, -0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, -0.0, 0.0, 1.0, -0.0, 0.0, 1.0, -0.0, 0.0, 1.0, -0.0, 0.0]


In [29]:
# display penalty vector s
s_opt = [s[i].x for i in range(m)]
s_opt


[14.334, 0.0, 0.0, 0.0, 0.0, 0.0]